<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Enforcing limits on primitives and computing missing quantities

## Authors: Samuel Cupp, Leo Werneck, and Zach Etienne

<font color='red'>**This module is currently under development**</font>

### Required and recommended citations:

* **(Required)** GRHayL (TBD)
* **(Required)** [Etienne, Paschalidis, Haas, Mösta, and Shapiro. IllinoisGRMHD: an open-source, user-friendly GRMHD code for dynamical spacetimes (2015)](http://arxiv.org/abs/1501.07276).

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This module is organized as follows

1. [**Introduction**](#introduction)
1. [**The `enforce_limits_on_primitives_and_compute_u0` function**](#enforce_limits_on_primitives_and_compute_u0)
    1. [*Function Arguments*](#fargs)
        1. [*Inputs*](#inputs)
        1. [*Outputs*](#outputs)
    1. [*Enforcing physical limits on the primitive variables*](#enforce_limits_on_primitives)
        1. [*Limits for the Hybrid EOS*](#hybrid_eos_limits)
        1. [*Limits for the Tabulated EOS*](#tabulated_eos_limits)
    1. [*Speed limit $v^i$ and compute $u^0$*](#speed_limit_v)

<a id='introduction'></a>

# Introduction \[Back to [top](#toc)\]
$$\label{introduction}$$

After the conservative-to-primitive routine runs, the primitives are not guaranteed to be within the EOS limits set by the simulation. For example, the returned $\rho_b$ may be less than $\rho_{\rm atm}$. The `enforce_limits_on_primitives_and_compute_u0` function enforces limits based on the EOS parameters. It also computes the zeroth component of the 4-velocity ($u^0$) while applying speed limits on $v^i$. This function naturally runs immediately before the `compute_conservs_and_Tmunu` [function](compute_conservs_and_Tmunu.ipynb).

<a id='enforce_limits_on_primitives_and_compute_u0'></a>

# The `enforce_limits_on_primitives_and_compute_u0` function \[Back to [top](#toc)\]
$$\label{enforce_limits_on_primitives_and_compute_u0}$$

<a id='fargs'></a>

## Function Arguments \[Back to [top](#toc)\]
$$\label{fargs}$$

First, let's look at the arguments of `enforce_primitive_limits_and_compute_u0`:
```
void enforce_primitive_limits_and_compute_u0(
      const GRHayL_parameters *restrict params,
      const eos_parameters *restrict eos,
      const metric_quantities *restrict metric,
      primitive_quantities *restrict prims,
      int *restrict speed_limit)
```

<a id='inputs'></a>

### Inputs
$$\label{inputs}$$

The parameters used from `params` are `evolve_entropy` and `psi6threshold`. The former controls whether the primitive entropy is computed. The latter triggers some of the limits.

Many parameters are used from `eos`. `eos_type` steers how the primitives are limited and calculated. `rho_min` and `rho_max` limit the value of $\rho_b$. `Gamma_th` is used to calculate $\epsilon$. `rho_atm` triggers some of the limits.

In addition, several functions are used from the `eos` struct. If the `eos_type` is hybrid, then the `hybrid_compute_P_cold_and_eps_cold` function is used. The `hybrid_compute_entropy_function` is also used if `evolve_entropy` is true.

The metric quantities used from `metric` are the shift (`betax`, `betay`, `betaz`), ADM metric (`adm_gxx`, `adm_gxy`, etc.), inverse lapse `lapseinv` $\left(\alpha^{-1}\right)$, inverse lapse squared `lapseinv2` $\left(\alpha^{-2}\right)$, and conformal factor quantity `psi6` ($\psi^6$). Since auxiliary quantities are used, we recommend use either `GRHayL_enforce_detgtij_and_initialize_metric` or `initialize_metric` to automatically compute these quantities from the ADM quantities.

The function expects the following primitive quantities in `prims` to be set: baryonic density `rho`, pressure `press`, and 3-velocity (`vx`, `vy`, `vz`).

<a id='outputs'></a>

### Outputs
$$\label{outputs}$$

The primitive quantities set in `prims` are the baryonic density `rho`, pressure `press`, 3-velocity (`vx`, `vy`, `vz`), $\epsilon$ `eps`, zeroth component of the 4-velocity `u0`. Additionally, `entropy` is set if `evolve_entropy` is true.

The argument `speed_limit` records if a speed limit is applied to $v^i$. The `con2prim_diagnostics` struct can record this by passing the `failure_checker` element. A value of 1000 is added to the speed_limit variable.

<a id='enforce_limits_on_primitives'></a>

## Enforcing physical limits on the primitive variables \[Back to [top](#toc)\]
$$\label{enforce_limits_on_primitives}$$

We first apply the simplest limits by enforcing that
$$
\rho_\mathrm{min} <= \rho_b <= \rho_\mathrm{max}
$$

We then apply the limits on pressure. Since hybrid and tabulated EOS require different treatment, the code splits between the two. 

<a id='hybrid_eos_limits'></a>

### Limits for the Hybrid EOS \[Back to [top](#toc)\]
$$\label{hybrid_eos_limits}$$

For hybrid EOS, if

$$
P < 0.9P_\mathrm{cold}
$$

then the pressure is floored to

$$
P = 0.9P_\mathrm{min}
$$

The factor of 0.9 helps with stability, as setting the condition to be exactly at $P_\mathrm{cold}$ can cause undesirable increases in the star pressure that behave better by allowing the pressure to fall slightly below $P_\mathrm{cold}$.

On the other end, the upper limit depends on the metric. If $\psi^6$ is greater than the GRHayL parameter $\psi^6_\mathrm{threshold}$, then


$$
P_\mathrm{max} = 10^5 P_\mathrm{cold}
$$

Otherwise,

$$
P_\mathrm{max} = 100 P_\mathrm{cold}
$$

The resulting limit is if

$$
P>P_\mathrm{max}~\mathrm{and~either}~\rho<100 \rho_\mathrm{atm}~\mathrm{or}~\psi^6 > \psi^6_\mathrm{threshold}
$$

then

$$
P = P_\mathrm{max}
$$

Summarizing the limits for the pressure,

<!---
I couldn't get the markdown table to do the rowspan/colspan, so I resorted to html. I am sorry.
--->
 <table>
  <tr style="border-bottom:1px solid black">
    <th style="border-right:1px solid black"></th>
    <th style="border-right:1px solid black; border-bottom:1px solid black; text-align:center">Lower Limit</th>
    <th colspan="2" style="text-align:center">Upper Limit</th>
  </tr>
  <tr>
    <th rowspan="2" style="border-right:1px solid black; border-bottom:1px solid black">Trigger</th>
    <td rowspan="2" style="border-right:1px solid black">$$P < 0.9P_\mathrm{min}$$</td>
    <th style="border-right:1px solid black">$$\psi^6 > \psi^6_\mathrm{threshold}$$</th>
    <th>$$\rho<100 \rho_\mathrm{atm}$$</th>
  </tr>
  <tr style="border-bottom:1px solid black">
    <td style="border-right:1px solid black">$$P > 10^5 P_\mathrm{cold}$$</td>
    <td>$$P > 100 P_\mathrm{cold}$$</td>
  </tr>
  <tr>
    <th style="border-right:1px solid black">Limit</th>
    <td style="border-right:1px solid black">$$P = 0.9P_\mathrm{min}$$</td>
    <td style="border-right:1px solid black">$$P = 10^5 P_\mathrm{cold}$$</td>
    <td>$$P = 100 P_\mathrm{cold}$$</td>
  </tr>
</table>

After setting $\rho_b$ and $P$, $\epsilon$ is computed. Entropy is also computed if `evolve_entropy` is true.

<a id='tabulated_eos_limits'></a>

### Limits for the Tabulated EOS \[Back to [top](#toc)\]
$$\label{tabulated_eos_limits}$$

TODO: add documentation!

<a id='speed_limit_v'></a>

## Speed limit $v^i$ and compute $u^0$ \[Back to [top](#toc)\]
$$\label{speed_limit_v}$$

Finally, we use the `limit_v_and_compute_u0` function (TODO: link to doc) to speed limit $v^i$ and compute $u^0$ regardless of which EOS being used.